# Data preparation for machine learning

By: Jacobo Fernandez-Vargas and Luca Citi

Based on the [scikit-learn](http://scikit-learn.org/stable/tutorial/basic/tutorial.html) and [pandas](https://pandas.pydata.org/docs/getting_started/index.html) documentation

## Loading data with Pandas

Pandas is a powerful library used in data analysis. We are not going to explore all its options but only focus on those functionalities that allow us to read data that contains non-numeric values such as text or dates. Pandas can also load data from excel files using the library 'xlrd'. Pandas supports reading and writing other complex data types such as sql, json, html and many more.
When reading CSV files, Pandas tries to infer the type from the values in each column. We can force Pandas to treat a column as containing dates and even specify that we want to treat dates as 'dayfirst', i.e. DD/MM/YYYY (the way used in several countries, including the UK) rather than MM/DD/YYYY (like in the USA).
If we have all numeric values in a CSV format we can also use Numpy directly to read the table but we will not have access to Pandas functionalities.

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('example1.csv', parse_dates=['Date'], dayfirst=True)
data

Just like `ndarray` is the fundamental brick of the numpy library, the dataframe is the fundamental brick of the pandas library.
A dataframe is a a table where each column is a variable (or series as they are called in Pandas) and each row is a sample. We can access each series using the name of the variable, either using the index notation (`df['v']`) or as field of the object (`df.v`). The latter notation can only be used when the field name is a valid field name (e.g., column '3a' can only be accessed as `df['3a']`).

The method `head` shows only the first 5 entries of the table.

In [ ]:
print(data['Date'].head())
print(data.Date.head())

To select rows, we ca use either a conditional format or a more traditional indexing.
For example let's select the rows for which the color is blue.

In [ ]:
data[data["Colour"] == 'Blue']

Now let's select the first five rows.

In [ ]:
data.iloc[:5,:]

Pandas is a really powerful tool but in the interest iof time we are not going to explain it further. However you can visit these [tutorials](https://pandas.pydata.org/docs/getting_started/index.html) to learn more about it.

## Prepare the data

Now that we have the data in the dataframe we will process it to transform all its values into numerical ones.
There are mainly two types of data that may be necessary to transform: dates and categorical values.
We can see what type of data we have in our dataframe:

In [ ]:
data.dtypes

(Note that the methods we see in this section replace non-numerical values with floating point numbers _in place_, so if you run them again you will get an error since the column does not contain the expected type anymore.)

### Dates

A standard way to transform dates is to change them into the time passed since a specific reference time point. This is as simple as:

In [ ]:
time0 = pd.Timestamp('2020-08-01 00:00:00')
data['Date'] = (data['Date'] - time0).values.astype(float)
data.head()

### Categorical values

Categorical values are non numerical values that may have or not an ordinal relationship. If they have an ordinal relationship (for example: low, mid, high or small, big), a simple approach is to transform directly to numerical value respecting the order (although this will impose an arbitrary distance between the values).

Pandas allows use to do this using a dictionary and the method `replace`.

In [ ]:
cleanup = {"Size": {"Small":0, "Medium":1, "Large":2}}
data.replace(cleanup, inplace=True)
data.head()

This method should not be used with categorical values with no ordinal relationship, for example the colour. Is red bigger than blue?


If we treat them as having ordinal relationship some algorithms may be unaffected by this. However, most algorithms will suffer (in terms of accuracy) when using this practice.
The 'proper' way to deal with such values is to create a number of variables equal to the the number of different values that the categorical value may take, and then set all to 0 except for one that will set to 1. This is called 'one-hot' encoding. In our case it would be somthing like:

Red -> \[0, 0, 1\]

Blue -> \[1, 0, 0\]

Green -> \[0, 1, 0\]

Fortunately, pandas makes this easy with the method `get_dummies`.

In [ ]:
data = pd.get_dummies(data, columns= ['Colour'])
data.head()

We can now confirm that all columns have been converted to numeric types.

In [ ]:
data.dtypes

Finally we want to transfer our data to two numpy arrays, one containing the variables (features) and another the labels (ground truth).

In [ ]:
import numpy as np
x = data.loc[:, data.columns != 'Label'].to_numpy()
y = data.loc[:, 'Label'].to_numpy()
print(x.shape, y.shape)

## Data Visualization

It is good practice to visualize the data to see if there is something obviously wrong. The matplotlib library includes a number of useful visualisation tools to help us. Getting familiar with this library (or a similar one) and its functions is a first step become a data scientist. Among the many functions make sure you become familiar with scatter, plot, and bar!

### False predictors

In some ocassions we may have a variable that is a false predictor, meaning that it's been introduced in the data by mistake and contains information about the label. Let's imagine that we have a dataset of clients of a bank, and we want to predict if a client will buy an add-on insurance. One of the variables could be how much the client pays to the bank for its services. If this price includes the cost of the insurance, this will result in a variable that may have a strong prediction power but won't work during the deployment because we will detect only the clients that already have an insurance.

In our data there is one false predictor, could you point which one looking at the plots?

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
for i in range(x.shape[1]):
    plt.subplot(3, 3, i+1)
    plt.scatter(y,x[:,i])
    plt.xticks([0, 1])
    plt.title(data.columns[i])

## Cross-validation
Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model may memorise the training data perfectly and have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called overfitting. To avoid it, when performing a (supervised) machine learning task, it is common practice to hold out part of the available data as a test set, which we will store in the variables `X_test` and `y_test`. 

Conceptually, we need to simulate the future situation, where we have trained our system with some data and, then, we apply it to some unseen data. However, to get better estimates of the accuracy of our model on new data, it is good practice to repeat the procedure after rotating which part of the data are used for train and which for testing. One such methos is called *cross-validation* and helps reduce the variability in estimating the accuracy on new data.

Scikit-learn is a library for machine learning (more on this in the next lab). It has a collection of classes which can be used to generate lists of train/test indices for popular cross-validation strategies.

They expose a `split` method which accepts the input dataset to be split and yields the train/test set indices for each iteration of the chosen cross-validation strategy.

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

k_fold = KFold(n_splits=5)
for train_indices, test_indices in k_fold.split(x):
    print('Train: %s | test: %s' % (train_indices, test_indices))

Graphically, we can represent the cross validation loop as follows (where 'O' reprersents a training sample and '•' a test one):

In [ ]:
for train_indices, test_indices in k_fold.split(x):
    for i in range(len(x)):
        print('O' if i in train_indices else '•', end='')
    print('')

Cross-validation is the cornerstone of supervised machine learning design. All the data-dependent manipulations (not only classification, but also pre-processing) must happen inside Cross-Validation loops when analysing the performance of a system.

Multiple cross-validation loops can be nested when performing hyper-paramether fitting. We will explore this in the next lab. See this [link](https://stats.stackexchange.com/q/292179) for a graphical representation of nested cross-validation.

The `Kfold` class in sklearn, has another paramether, shuffle. Read the documentation about it and explore what changes when its value is True.

## Pre-processing

### Missing data

It is common to have missing values in some of the variables. We can see this counting the number of NaN (Not a Number) in our dataset:

In [ ]:
np.sum(np.isnan(x), 0)

Here we can see that the second variable in our dataset has some values that are missing.

#### Default value

The fastest and easiest way to deal with missing data, is to assing a default value (ex. 0). This is possible if we know something about the variable and which may be a good default value, but sometimes we don't.

#### Data-driven value

A better option, is to use a value that is derived from the data, such as the mean, median or other statistical value. However, as mentioned before, this should happen inside the cross-validation (i.e. any value used for imputation must be computed using only the training data).

Here we calculate the mean of the second variable in our dataset for different folds. We use the function `nanmean` from numpy to calculate the mean of an array containing NaN values.

In [ ]:
k_fold = KFold(n_splits=5)
for train_indices, test_indices in k_fold.split(x):
    print(np.nanmean(x[train_indices, 1]))

As you can see the value of that mean differs for each fold. This is why is important to do it inside the fold validation loop.

### Data transformation

Many algorithms work better when the data are inside a specific range, or when all the data have similar ranges. This is not the case for all the algorithms (some work equally well with unnormalised or normalised data).

#### Data-clipping

One option is to just set predefined maximum and minimum values. For this, we can use the `clip` method from numpy.

#### Scaling and Normalization

In the same way, as with the missing data, a better practice is to perfform this procedure in a data-driven manner. For this, sklearn provides several options in its 'preprocessing' package. As before, this normalisation step should be included inside the cross validation loop as well.

In [ ]:
from sklearn.preprocessing import StandardScaler
k_fold = KFold(n_splits=5)
x_or = x.copy() # We make a copy of our data to avoid overwriting
for train_indices, test_indices in k_fold.split(x):
    mean = np.nanmean(x_or[train_indices, 1])
    x[np.isnan(x_or[:,1]), 1] = mean # We substitute the missing values for the mean
    ss = StandardScaler()
    ss.fit(x[train_indices, :])
    x_train = ss.transform(x[train_indices, :])
    x_test = ss.transform(x[test_indices, :])

In this example we see something that is common in many methods in the sklearn package, the division between `fit` and `transform`. These allow to the creation of pipelines which help streamline the modeling of different methods.

## Exercise

Using what you have learned in this notebook, prepare the data from the file 'lizards.csv'. Note that in this case you need to manipulate the 'Label' column as well.

### Extra 1

For the lizards dataset, the columns 'Date' and 'Hour' could be merged, into a single value (even directly upon reading the CSV file). Explore how to do it either in Pandas or Numpy.

### Extra 2

This file indicates the position (East, North) where some lizards were seen. Each lizard has an ID (ID_e) and has been spotted several times. To infeer the position for those samples where there is no available data, instead of using the mean (or any other imputation method) based on the whole training set, try to calculate the mean (or other statistical value) for that specific lizard based on the training data. Note that, depending on how you divide the data, there could be unknown data in the test set for a lizard that was not included in the training data. Think of a possible way to deal with this issue.

## Further reading on Pandas

Pandas is a key python library for datascience.  There's a good youtube tutorial on it here https://www.youtube.com/watch?v=dPwLlJkSHLo 